In [2]:
from pprint import pprint
from Parser import Parser
import util
from tfidf import *
import glob, os
import math
import numpy as np
from numpy.linalg import norm
from tqdm import tqdm
from __future__ import division, unicode_literals
from textblob import TextBlob as tb

class VectorSpace:
    """ A algebraic model for representing text documents as vectors of identifiers. 
    A document is represented as a vector. Each dimension of the vector corresponds to a 
    separate term. If a term occurs in the document, then the value in the vector is non-zero.
    """

    #Collection of document term vectors
    documentVectors = []

    #Mapping of vector index to keyword
    vectorKeywordIndex=[]

    #Tidies terms
    parser=None
    
    def __init__(self, documents=[], vectorMode = 'tf'):
        self.documentVectors=[]
        self.parser = Parser()
        self.BlobList = self.getBlobList(documents)
        self.vectorMode = vectorMode
        if(len(documents)>0):
            self.build(documents)
    
    def getBlobList(self, documents): 
        bloblist = []
        for doc in documents:
            wordList = self.parser.tokenise(doc)
            wordList = self.parser.removeStopWords(wordList)
            bloblist.append(tb(" ".join(wordList)))
        return bloblist
            
    def build(self,documents):
        """ Create the vector space for the passed document strings """
        self.vectorKeywordIndex = self.getVectorKeywordIndex(documents)
        self.documentVectors = [self.makeVector(document, self.vectorMode) for document in tqdm(documents)]

    def getVectorKeywordIndex(self, documentList):
        """ create the keyword associated to the position of the elements within the document vectors """

        #Mapped documents into a single word string	
        vocabularyString = " ".join(documentList)

        vocabularyList = self.parser.tokenise(vocabularyString) #斷詞
        #Remove common words which have no search value
        vocabularyList = self.parser.removeStopWords(vocabularyList) # 去掉stop words
        uniqueVocabularyList = util.removeDuplicates(vocabularyList) # 去掉重複詞
        
        vectorIndex={}
        offset=0
        #Associate a position with the keywords which maps to the dimension on the vector used to represent this word
        for word in uniqueVocabularyList:
            vectorIndex[word]=offset
            offset+=1
        return vectorIndex  #(keyword:position)


    def makeVector(self, wordString, mode):
        """ @pre: unique(vectorIndex) """
        #Initialise vector with 0's
        vector = [0] * len(self.vectorKeywordIndex)
        wordList = self.parser.tokenise(wordString)
        wordList = self.parser.removeStopWords(wordList)
        tbString = tb(" ".join(wordList))
        if mode == 'tf':
            for word in list(set(wordList)):
                vector[self.vectorKeywordIndex[word]] = tf(word, tbString) #Use simple Term Count Model
            return vector 
        
        if mode == 'tf-idf':
            #print('bloblist:', self.BlobList)
            for word in list(set(wordList)):
                vector[self.vectorKeywordIndex[word]] =  tfidf(word, tbString , self.BlobList) 
            return vector
        
    def buildQueryVector(self, termList):
        """ convert query string into a term vector """
        query = self.makeVector(" ".join(termList), self.vectorMode)
        return query

    def related(self,documentId):
        """ find documents that are related to the document indexed by passed Id within the document Vectors"""
        ratings = [util.cosine(self.documentVectors[documentId], documentVector) for documentVector in self.documentVectors]
        #ratings.sort(reverse=True)
        return ratings
    
    def search(self,searchList, mode = 'cos'):
        """ search for documents that match based on a list of terms """
        if type(searchList[0]) == str:
            queryVector = self.buildQueryVector(searchList)
        else:
            queryVector = searchList
        
        if mode == 'cos':
            ratings = [util.cosine(queryVector, documentVector) for documentVector in tqdm(self.documentVectors)]
        #ratings.sort(reverse=True)
            return ratings
        if mode == 'eucli':
            ratings = [util.Euclidean(queryVector, documentVector) for documentVector in tqdm(self.documentVectors)]
            return ratings

    def printresult(self,searchlist, files,n,mode='cos'):
        scoreList = self.search(searchlist, mode = mode)
        for i in np.flip(np.argsort(scoreList))[:n]:
            print( 'document:' , files[i],'score:', scoreList[i])
        return np.flip(np.argsort(scoreList))[:n]
            
    def getFeedbackVector(self, searchList, top10results ,mode = 'cos'):
        queryVector = self.buildQueryVector(searchList)
        feedbackVector = self.buildQueryVector(top10results)
        queryArray = np.array(queryVector)
        feedbackArray = np.array(feedbackVector)
        newQueryVector = list(queryArray + 0.5 * feedbackArray)
        return newQueryVector

In [3]:
# Q3
documents_chi = [] #0-499是英文檔案 ， 500-1500是中文檔案
files_chi = []
for file in os.listdir("./News/News"):
    if file.endswith(".txt"):
        filename_chi = os.path.join("./News/News", file)
        files_chi.append(file[:-4])
        with open(filename_chi, encoding="utf-8") as f:
            lines = f.readlines()
            doc = ' '.join(lines)
            doc1 = doc.replace("\n", "")
            documents_chi.append(doc1)

In [6]:
# Q3
documents_chi = [] #0-499是英文檔案 ， 500-1500是中文檔案
files_chi = []
for file in os.listdir("./News/News"):
    if file.endswith(".txt"):
        filename_chi = os.path.join("./News/News", file)
        files_chi.append(file[:-4])
        with open(filename_chi, encoding="utf-8") as f:
            lines = f.readlines()
            doc = ' '.join(lines)
            doc1 = doc.replace("\n", "")
            documents_chi.append(doc1)

In [16]:
print(len(documents_chi[500:1500]))

1000


In [53]:
#test data

documents = ["The cat cat in the hat disabled",
                 "A cat is a fine pet ponies.",
                 "Dogs and cats make good pets.",
                 "I haven't got a hat.",
            "烏克蘭總統是澤倫斯基。", 
            "澤倫斯基是親西方國家的烏克蘭總統，這讓普丁很不爽"]

vectorSpace = VectorSpace(documents, 'tf-idf')  # vectorSpace(documents, vectorMode = 'tf' or 'tf-idf') (default is 'tf')


#print(vectorSpace.vectorKeywordIndex)

#print(vectorSpace.documentVectors)

#print(vectorSpace.related(1))

#print(vectorSpace.search(["cat"]))   

vectorSpace.printresult(["cat"])
vectorSpace.printresult(["cat"], mode = 'eucli')  # mode = 'cos' or 'eucli' (default is 'cos')

###################################################

score: nan document: I haven't got a hat.
score: nan document: Dogs and cats make good pets.
score: nan document: A cat is a fine pet ponies.
score: nan document: The cat cat in the hat disabled
score: 1.2345525572306575 document: Dogs and cats make good pets.
score: 1.0216002166437488 document: A cat is a fine pet ponies.
score: 0.7504758415354574 document: The cat cat in the hat disabled
score: 0.28768207245178085 document: I haven't got a hat.
